In [284]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [285]:
# Echarts Jupyter lab support
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

In [286]:
# Echarts imports
from pyecharts import options as opts
from pyecharts.globals import  ThemeType
from pyecharts.charts import Line

In [287]:
from IPython.display import HTML
import matplotlib.pyplot as plt

In [288]:
from common.log_helper import LogHelper
from backtest.scale_backtest_args import ScaleBacktestArgGen
from backtest.scale_backtest import ScaleBacktest
from modelservice.alpha_consts import SignalDirection
from notebooks.echart_bt_plots import *
from notebooks.echart_df_plot import *
from copy import deepcopy
import concurrent.futures
import time
import os
import requests
import json
import logging
import sys
import pandas as pd

In [289]:
#LogHelper.configure_logging(verbose=False)
pd.set_option("display.precision", 2)
pd.set_option('display.float_format', '{:.2f}'.format)

In [290]:
BtArgs : dict = {
    'common': {
        'start_time': '09:15:01',
        'end_time' :  '15:24:30',
    },
    'alpha': {
        'sigmult' : None,
        'smoother' : None,
    },
    'execution': {
        'exec_instr_type' : 'NK_F',        
        'price_type' : 'CROSS',
        'opt_trd_offset' : 2,
        'quote_lots' : 1,
        'stop_loss': 0,
        'max_loss' : 0,
        'profit_target' : 0
    }
}

In [291]:
dump_base_dir = "/home/qoptrprod/data/dumps/"
underlyings = ["NIFTY_I","BANKNIFTY_I","FINNIFTY_I","MIDCPNIFTY_I"]
#underlyings = ["NIFTY_I"]
trading_dates = [20240319, 20240320, 20240321, 20240322, 20240326, 20240327,
                 20240328, 20240401, 20240402, 20240403, 20240404, 20240405,
                 20240408, 20240409, 20240410, 20240412, 20240415, 20240416,
                 20240418, 20240419
                ]
#trading_dates = [20240321,20240328,20240404,20240418]
trading_dates = [str(t) for t in trading_dates]
stop_loss = -7000
sig_mults = [0.5]
smoothers = [
        {
            'TS' : {'enabled': False, 'type' : 'STABLEWIN' , 'timeperiod' : 0 },
            'UL' : {'enabled': True, 'type' : 'EMA' , 'timeperiod' : 30 },
            'CP' : {'enabled': True, 'type' : 'SMA' , 'timeperiod' : 30 },            
        },
        {
            'TS' : {'enabled': False, 'type' : 'STABLEWIN' , 'timeperiod' : 0 },
            'UL' : {'enabled': True, 'type' : 'EMA' , 'timeperiod' : 3 * 60 },
            'CP' : {'enabled': True, 'type' : 'SMA' , 'timeperiod' : 3 * 60 },            
        },
        {
            'TS' : {'enabled': False, 'type' : 'STABLEWIN' , 'timeperiod' : 0 },
            'UL' : {'enabled': True, 'type' : 'EMA' , 'timeperiod' : 30 },
            'CP' : {'enabled': True, 'type' : 'SMA' , 'timeperiod' : 30 },            
            'MACD' : {'enabled': True, 'dfast' : 2*60, 'dslow' : 5*60, 'dsig' : 3*60, 'timeperiod' : 'macd1'  },                        
        },
        {
            'TS' : {'enabled': False, 'type' : 'STABLEWIN' , 'timeperiod' : 0 },
            'UL' : {'enabled': True, 'type' : 'EMA' , 'timeperiod' : 3 * 60 },
            'CP' : {'enabled': True, 'type' : 'SMA' , 'timeperiod' : 3 * 60 },            
            'MACD' : {'enabled': True, 'dfast' : 2*60, 'dslow' : 5*60, 'dsig' : 3*60, 'timeperiod' : 'macd1'  },                        
        },
        {
            'TS' : {'enabled': False, 'type' : 'STABLEWIN' , 'timeperiod' : 0 },
            'UL' : {'enabled': True, 'type' : 'EMA' , 'timeperiod' : 3 * 60 },
            'CP' : {'enabled': True, 'type' : 'SMA' , 'timeperiod' : 3 * 60 },            
            'MACD' : {'enabled': True, 'dfast' : 5*60, 'dslow' : 13*60, 'dsig' : 8*60, 'timeperiod' : 'macd2'},
        }
   ] 

In [292]:
def bt_wrap(arg_obj):
    bt = ScaleBacktest(arg_obj)
    scale_result = bt.run_bt(show_trades=False)
    return scale_result

In [ ]:
results = []
with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
    fmaplist = []
    for underlying in underlyings:
        for trading_date in trading_dates:
            for smoother in smoothers:
                for sigmult in sig_mults:            
                    bt_args = deepcopy(BtArgs)
                    bt_args['alpha']['sigmult'] = sigmult
                    bt_args['alpha']['smoother'] = smoother
                    enab_flag = ''.join([str(smoother[k]['enabled'])[0] for k in smoother.keys()])
                    ts_valstr = '_'.join([str(smoother[k]['timeperiod']) for k in smoother.keys()])
                    key = f"mult={sigmult}~dur={enab_flag}_{ts_valstr}"
                    arg_obj = ScaleBacktestArgGen.resolve_and_gen_args(dump_base_dir,underlying,trading_date=trading_date,bt_args=bt_args)
                    fmaplist.append(
                        {
                            'a.date' : trading_date,
                            'b.underlying' : underlying,
                            'c.key' : key,
                            'process' : executor.submit(bt_wrap, arg_obj)
                        })
    
    for fmap in fmaplist:
        sdr = {}
        for k,v in fmap.items():
            if k == 'process':
                result = v.result()
                for sk, sv in result['summary'].items():
                    sdr[sk] = sv
            else:
                sdr[k] = v
        results.append(sdr)
# process results               


In [294]:
sdf = pd.DataFrame(results, columns=sorted(results[0].keys()))
sdf = sdf.drop(columns=['to'])
sdf = sdf.rename(columns={'avg_entry_px' : 'apx', 'max_hold_sec' : 'hold', 'num_trades' : 'nt', 'quote_lots' : 'ql'})
sdf['orig_net_pnl'] = sdf['net_pnl']
sdf.loc[sdf['net_pnl'] < stop_loss, 'net_pnl'] = stop_loss
sdf['SLFail'] = sdf['orig_net_pnl'] > sdf['net_pnl']
sdf['day'] = pd.to_datetime(sdf['a.date'], format='%Y%m%d').dt.day_name()

In [295]:
#display(sdf)

### Summary

In [296]:
import numpy as np
def days(l):
    return len(l)
def neg(l):
    return (l < 0).sum()
def stop_loss_fails(l):
    return (l == True).sum()


display(
sdf.groupby(['b.underlying']).agg({
    'net_pnl' : ['sum', days, neg, stop_loss_fails, 'median', 'mean'],
    'tcost':'sum', 
    'max_loss' : 'min', 
    'max_profit' : 'max',
    'nt' : 'sum',
    'lot_size' : 'max',
    'apx' : 'max',
}))    

pnls = sdf.net_pnl.values
print(f"   Total PNL: {round(sum(pnls))} in {len(trading_dates)} days")
print(f"Avgerage PNL: {round(sum(pnls)/len(trading_dates))} per day")

net_pnl                                                tcost  \
                    sum days neg stop_loss_fails   median     mean       sum   
b.underlying                                                                   
BANKNIFTY_I   -51876.30  100  63               0  -966.50  -518.76  52934.90   
FINNIFTY_I   -175657.60  100  61               0 -1524.30 -1756.58  42324.60   
MIDCPNIFTY_I -219161.80  100  71               0 -3825.85 -2191.62 103015.50   
NIFTY_I         5871.40  100  52               0  -163.15    58.71  56549.40   

              max_loss max_profit    nt lot_size    apx  
                   min        max   sum      max    max  
b.underlying                                             
BANKNIFTY_I   -9604.50   14540.25   574       15  48966  
FINNIFTY_I   -16130.00   14952.00   402       40  21788  
MIDCPNIFTY_I  -9776.25   12885.00  1028       75  10973  
NIFTY_I       -9345.00   10985.00   432       50  22805

   Total PNL: -440824 in 20 days
Avgerage PNL: -22041 per day


In [297]:
sdf.to_csv("output_ticker.csv", index=False)